In [1]:
!pip install opendatasets

In [2]:
import opendatasets as od
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

In [3]:
# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [4]:

import opendatasets as od
import pandas

od.download(
	"https://www.kaggle.com/datasets/elemento/nyc-yellow-taxi-trip-data")


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: pavanesh1
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/elemento/nyc-yellow-taxi-trip-data


In [9]:
df = pd.read_csv('nyc-yellow-taxi-trip-data/yellow_tripdata_2016-01.csv', nrows=10000)  # 10k rows for speed

In [32]:
# Simulate balanced taxi reviews
def generate_fake_review(row):
    pickup = pd.to_datetime(row['tpep_pickup_datetime'])
    dropoff = pd.to_datetime(row['tpep_dropoff_datetime'])
    duration = (dropoff - pickup).total_seconds() / 60  # Duration in minutes
    fare = row['fare_amount']
    if duration > 15 or fare > 10:  # Adjusted for more negatives
        return np.random.choice([
            "This taxi was too slow and overpriced!",
            "Terrible ride, took way too long!",
            "Waste of money, awful service!",
            "Driver was bad, trip was a mess!"
        ])
    else:
        return np.random.choice([
            "Fast ride, great driver!",
            "Awesome taxi, worth every penny!",
            "Quick trip, excellent service!",
            "Smooth ride, got there early!"
        ])

In [47]:
df['review'] = df.apply(generate_fake_review, axis=1)
df['sentiment'] = np.where((df['fare_amount'] > 10) |
                          ((pd.to_datetime(df['tpep_dropoff_datetime']) -
                            pd.to_datetime(df['tpep_pickup_datetime'])).dt.total_seconds() / 60 > 15),
                          'negative', 'positive')

In [48]:
print("Sentiment distribution:")
print(df['sentiment'].value_counts())

Sentiment distribution:
sentiment
positive    5860
negative    4140
Name: count, dtype: int64


In [50]:
df['processed_text'] = df['review'].apply(preprocess_text)

In [51]:
X = df['processed_text']
y = df['sentiment']

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [53]:
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [54]:
# Train model
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

LogisticRegression()

In [56]:
# Evaluate
accuracy = accuracy_score(y_test, y_pred)
print(f"\nAccuracy: {accuracy:.2f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 1.00

Classification Report:
              precision    recall  f1-score   support

    negative       1.00      1.00      1.00       835
    positive       1.00      1.00      1.00      1165

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000



In [58]:
# Test with multiple samples
test_reviews = [
    "Amazing drive, reached early!",
    "Drive was bad, too slow!",
    "This taxi was fast and cheap!",
    "Terrible trip, waste of time!",
    "The driver was awful and late!"
]
for review in test_reviews:
    print(f"\nSample review: {review}")
    print(f"Predicted sentiment: {predict_sentiment(review)}")


Sample review: Amazing drive, reached early!
Predicted sentiment: positive

Sample review: Drive was bad, too slow!
Predicted sentiment: negative

Sample review: This taxi was fast and cheap!
Predicted sentiment: positive

Sample review: Terrible trip, waste of time!
Predicted sentiment: negative

Sample review: The driver was awful and late!
Predicted sentiment: negative
